In [1]:
import pandas as pd

In [8]:
df_s_3 = pd.read_pickle('df_s_3_1.pkl')

In [11]:
df_s_4 = df_s_3.groupby(['contract','ProductId']).agg({
                                        'Volume' : 'sum',
                                        'ContractVolume':'mean',
                                        'contractValue' : 'mean',
                                        'contractValue_adj': 'sum',
                                        'volByquote' : 'sum'
                                   }).reset_index()

In [12]:
df_s_5 = df_s_4.groupby(['contract']).agg({
                                        'Volume' : 'sum',
                                        'ContractVolume':'sum',
                                        'contractValue' : 'sum',
                                        'contractValue_adj': 'sum',
                                        'volByquote' : 'sum'
                                   }).reset_index()

In [13]:
df_s_5.query('contract == "4001332502"')

contract   Volume  ContractVolume  contractValue  contractValue_adj  \
37  4001332502  186.487           162.0        15274.8         17605.8726   

      volByquote  
37  17779.244203

In [14]:
df_m_2 = pd.read_pickle('df_m_2.pkl')

In [15]:
df_m_2.columns

Index(['plantcode', 'Material#', 'grade', 'CaseStatus', 'SAPOrderNo',
       'CaseOwner', 'Sold-tocustomer', 'Ship-tocustomer', 'Plant',
       'ShippingCondition', 'ContractStart', 'ContractEnd', 'Material',
       'ContractVolume', 'SpotPrice', 'Signal', 'IsLatestCycle',
       'Datetimerounded', 'region'],
      dtype='object')

In [16]:
df_omr = df_m_2[['SAPOrderNo','Material#','region','grade','ContractStart']]

df_a_3 = pd.read_pickle('df_a_3.pkl')

df_omr_2 = pd.merge(df_omr,df_a_3[['Quote date', 'quote', 'region', 'Grade ']],
                  how='left',left_on=['ContractStart','grade','region'],right_on=['Quote date','Grade ','region'])

# df_omr_2.isnull().sum()

# df_omr_2.shape

df_omr_2 = df_omr_2.drop_duplicates(subset = ['SAPOrderNo','Material#'])

# df_omr_2.columns

df_omr_2.rename(columns={'quote':'contract_st_omr_qt'},inplace=True)

# df_s_3.isnull().sum()

df_s_3_1 = pd.merge(df_s_3,df_omr_2[['SAPOrderNo', 'Material#','contract_st_omr_qt']],
                    how='left',left_on=['contract','Material#'],right_on=['SAPOrderNo','Material#'])

# df_s_3_1.isnull().sum()

# df_s_3_1

In [35]:
df_s_3_1.columns

Index(['contract', 'Products', 'Date', 'Volume', 'ProductId', 'grade',
       'Material#', 'SAPOrderNo_x', 'region', 'ContractVolume', 'SpotPrice',
       'Quote date', 'quote', 'Grade ', 'contract_st_omr_qt'],
      dtype='object')

In [34]:
df_s_3_1.drop(['SpotPrice_adj', 'contractValue',
       'volByquote', 'contractValue_adj', 'SAPOrderNo_y'],axis=1,inplace=True)

In [36]:
df_s_3_1.columns

Index(['contract', 'Products', 'Date', 'Volume', 'ProductId', 'grade',
       'Material#', 'SAPOrderNo_x', 'region', 'ContractVolume', 'SpotPrice',
       'Quote date', 'quote', 'Grade ', 'contract_st_omr_qt'],
      dtype='object')

In [46]:
df_s_3_1['omr_spot'] = df_s_3_1['contract_st_omr_qt']*10

df_s_3_1['contractValue_omr'] = df_s_3_1['omr_spot']*df_s_3_1['ContractVolume']
df_s_3_1['volByquote'] = df_s_3_1['Volume']*df_s_3_1['quote']*10
df_s_3_1['contractValue_adj'] = df_s_3_1['omr_spot']*df_s_3_1['Volume']

In [47]:
df_s_4_1 = df_s_3_1.groupby(['contract','ProductId']).agg({
                                        'Volume' : 'sum',
                                        'ContractVolume':'mean',
                                        'contractValue_omr' : 'mean',
                                        'contractValue_adj': 'sum',
                                        'volByquote' : 'sum'
                                   }).reset_index()

In [48]:
df_s_5_1 = df_s_4_1.groupby(['contract']).agg({
                                        'Volume' : 'sum',
                                        'ContractVolume':'sum',
                                        'contractValue_omr' : 'sum',
                                        'contractValue_adj': 'sum',
                                        'volByquote' : 'sum'
                                   }).reset_index()

In [54]:
df_s_5_1.query('contract == "4001332502"')

contract   Volume  ContractVolume  contractValue_omr  contractValue_adj  \
37  4001332502  186.487           162.0         152798.825      176143.713595   

       volByquote       Profit  
37  177792.442026 -1648.728431

In [50]:
df_s_5_1['Profit'] =  df_s_5_1.contractValue_adj - df_s_5_1.volByquote

In [51]:
df_s_5_1.query('contract == "4001367391"')

contract  Volume  ContractVolume  contractValue_omr  \
11933  4001367391  97.276           100.0           94803.65   

       contractValue_adj    volByquote     Profit  
11933       92221.198574  92929.252804 -708.05423

In [53]:
df_s_5_1.Profit.sum()

-1823758.9949585048

In [55]:
df_s_5_1.to_pickle('df_s_5_1.pkl')

df_s_3_1.to_pickle('df_s_3_1.pkl')

df_omr_2.to_pickle('df_omr_2.pkl')

In [2]:
df_s_5_1 = pd.read_pickle('df_s_5_1.pkl')

In [4]:
df_m_2 = pd.read_pickle('df_m_2.pkl')

In [5]:
df_m_3 = df_m_2.drop_duplicates(subset=['SAPOrderNo'],keep='first')

In [6]:
df_s_6 = pd.merge(df_s_5_1,df_m_3,how='left',left_on=['contract'],right_on=['SAPOrderNo'])

In [7]:
df_s_6.columns

Index(['contract', 'Volume', 'ContractVolume_x', 'contractValue_omr',
       'contractValue_adj', 'volByquote', 'Profit', 'plantcode', 'Material#',
       'grade', 'CaseStatus', 'SAPOrderNo', 'CaseOwner', 'Sold-tocustomer',
       'Ship-tocustomer', 'Plant', 'ShippingCondition', 'ContractStart',
       'ContractEnd', 'Material', 'ContractVolume_y', 'SpotPrice', 'Signal',
       'IsLatestCycle', 'Datetimerounded', 'region'],
      dtype='object')

In [8]:
df_s_6.drtableop(['ContractVolume_y'],axis=1,inplace=True)

In [9]:
df_s_5_1.shape

(12723, 7)

In [10]:
df_s_6.shape

(12723, 25)

In [28]:
df_s_6.to_excel('data_viz_3.xlsx')

In [12]:
# by contract duration analysis 
df_s_6['contract_duration']= (df_s_6.ContractEnd-df_s_6.ContractStart).dt.days

In [14]:
df_s_6.contract_duration.describe()

count    12723.000000
mean        13.338757
std          5.443381
min          0.000000
25%         11.000000
50%         13.000000
75%         14.000000
max         35.000000
Name: contract_duration, dtype: float64

In [25]:
bins_val = [0,7,14,21,28,100] 
bin_labels = ['0-7','7-14','14-21','21-28','>28']
df_s_6['contract_dur_bucket'] = pd.cut(df_s_6['contract_duration'], bins = bins_val,
                                   labels=bin_labels,include_lowest=True)

df_s_6.groupby('contract_dur_bucket')['Profit'].describe().unstack().reset_index()

contract_dur_bucket   count         mean          std           min  \
0                 0-7  1305.0    -8.174201   352.967294  -2757.299642   
1                7-14  8880.0  -139.795415  1170.656543 -28882.271113   
2               14-21  1389.0  -283.355350  2498.792698 -38821.479762   
3               21-28  1025.0  -304.751199  3612.290642 -47424.020429   
4                 >28   124.0  1082.598235  5520.616830 -12946.426928   

          25%         50%         75%           max  
0  -71.208150    0.000000   42.794230   4893.061225  
1 -332.112716  -31.578199  133.611928  13088.716056  
2 -651.685125 -106.941087  207.175410  14940.674248  
3 -829.772340 -153.583313  348.795720  42327.692680  
4 -689.376371 -203.484577  627.832993  30671.194840

In [17]:
df_s_6.groupby('contract_dur_bucket')['Profit'].quantile([0,0.05,0.25,0.30,.35,0.5,0.75,0.95,1]).unstack().reset_index()

contract_dur_bucket           0.0         0.05        0.25         0.3  \
0                 0-7  -2757.299642  -450.385824  -89.403260  -58.384244   
1                7-14 -28882.271113 -1569.847363 -332.112716 -248.235363   
2               14-21 -38821.479762 -3490.928271 -651.685125 -480.080125   
3               21-28 -47424.020429 -3854.336728 -829.772340 -587.893726   
4                 >28 -12946.426928 -3235.372103 -689.376371 -597.781206   

         0.35         0.5        0.75          0.95           1.0  
0  -30.796053    0.000000   59.448729    445.527437   4893.061225  
1 -182.207130  -31.578199  133.611928    996.258862  13088.716056  
2 -347.273199 -106.941087  207.175410   2502.006417  14940.674248  
3 -463.863832 -153.583313  348.795720   2576.632237  42327.692680  
4 -484.859711 -203.484577  627.832993  10217.804131  30671.194840

In [18]:
df_s_6.groupby('contract_dur_bucket')['Profit'].sum()

contract_dur_bucket
0-7     -1.066733e+04
7-14    -1.241383e+06
14-21   -3.935806e+05
21-28   -3.123700e+05
>28      1.342422e+05
Name: Profit, dtype: float64

In [31]:
df_s_6.groupby('CaseOwner')['Profit'].quantile([0,0.05,0.25,0.30,.35,0.5,0.58,0.75,0.95,1]).unstack().reset_index()

CaseOwner           0.0         0.05        0.25         0.3  \
0             Amelie Busse  -5128.198361  -707.998815 -105.314969  -78.216551   
1             Anna Kuegler  -2050.351433 -1374.172699 -204.943681 -198.070552   
2               Brit Erben  -6359.596497 -1758.268496 -403.169245 -309.506600   
3          David Jedrasiak   -418.789842  -386.549147 -257.586365 -225.345670   
4          Florian Seiwert -47424.020429 -3322.388728 -547.109235 -364.720894   
5         Friederike Klatt  -1516.277633  -782.188429  -98.624682  -72.726125   
6            Jacob Joerkel  -2291.487478  -514.757438 -165.335087  -85.850901   
7           Marion Mackert  -8484.571063 -2033.588277 -405.525027 -277.641016   
8           Michael Sehlke  -4991.708469 -1023.769079 -297.442704 -229.858320   
9    Peter Schimmelpfennig -11074.748802 -1496.308982 -323.592518 -250.714584   
10         Thomas Passmann -11595.394961 -1405.554085 -309.345120 -224.309340   
11          Torsten Feiler -22592.139869 -3692.928761 -771.692321 -530.946625   
12  Yildiz Pinarbasi-Mayer -10040.461113 -1486.579854 -405.006557 -308.712131   
13         Yvonne Buchholz  -6931.851452  -980.990096 -188.885871 -131.041628   

          0.35           0.5          0.58        0.75         0.95  \
0   -56.827687 -2.129400e+00  0.000000e+00  134.049025   605.076159   
1  -191.197423 -9.726491e+01 -7.541945e+01  -60.674745   358.531412   
2  -210.930607 -6.291363e+01 -1.818989e-12   73.420654   822.859614   
3  -193.104975 -9.638289e+01 -4.479778e+01   64.820588   193.783369   
4  -261.900128 -5.668023e+01  0.000000e+00  238.948762  2594.095379   
5   -64.080643 -3.498891e+01  0.000000e+00   30.236844   382.234195   
6   -36.295243  1.483124e+01  3.296136e+01  148.053422   382.846051   
7  -214.592647 -2.959815e+01  0.000000e+00  201.302322  1341.949769   
8  -164.085074 -3.288808e+01  0.000000e+00  122.973526   855.643603   
9  -183.912078 -3.983909e+01  0.000000e+00   59.972101   857.731535   
10 -161.207486 -2.728484e-12  2.299705e+01  202.735083  1141.312800   
11 -364.769473 -8.406584e+01  0.000000e+00  222.464861  2051.178489   
12 -226.886725  0.000000e+00  3.928475e+01  304.327849  1616.966819   
13  -90.649316 -4.078905e+00  0.000000e+00   59.396571   593.239749   

             1.0  
0    2288.438295  
1     597.648389  
2    9437.103647  
3     226.024064  
4   42327.692680  
5    1321.769449  
6     454.379737  
7   11496.042057  
8    6995.643798  
9    9236.771810  
10  14940.674248  
11  24353.990304  
12   8468.888961  
13  11854.034251

In [21]:
df_s_6.groupby('CaseOwner')['Profit'].sum().sort_values(0)

CaseOwner
Torsten Feiler           -478005.962834
Florian Seiwert          -360768.677633
Peter Schimmelpfennig    -335697.302554
Brit Erben               -203755.923434
Yvonne Buchholz          -154190.278806
Marion Mackert            -99690.609175
Michael Sehlke            -90575.088757
Thomas Passmann           -70355.204831
Yildiz Pinarbasi-Mayer    -11616.669330
Amelie Busse               -6339.751832
Friederike Klatt           -6306.039123
Jacob Joerkel              -3838.193600
Anna Kuegler               -2426.527274
David Jedrasiak             -192.765778
Name: Profit, dtype: float64

In [26]:
df_s_6.isnull().sum()

contract               0
Volume                 0
ContractVolume_x       0
contractValue_omr      0
contractValue_adj      0
volByquote             0
Profit                 0
plantcode              0
Material#              0
grade                  0
CaseStatus             0
SAPOrderNo             0
CaseOwner              0
Sold-tocustomer        0
Ship-tocustomer        0
Plant                  0
ShippingCondition      0
ContractStart          0
ContractEnd            0
Material               0
SpotPrice              0
Signal                 0
IsLatestCycle          0
Datetimerounded        0
region                 0
contract_duration      0
contract_dur_bucket    0
dtype: int64

In [24]:
df_s_6.loc[df_s_6.contract_dur_bucket.isnull(),['contract_duration','ContractStart','ContractEnd']]

contract_duration ContractStart ContractEnd
39                     0    2019-01-02  2019-01-02
40                     0    2019-01-02  2019-01-02
81                     0    2019-01-02  2019-01-02
103                    0    2019-01-03  2019-01-03
146                    0    2019-01-15  2019-01-15
191                    0    2019-01-02  2019-01-02
340                    0    2019-01-04  2019-01-04
630                    0    2019-01-10  2019-01-10
742                    0    2019-01-22  2019-01-22
743                    0    2019-01-14  2019-01-14
818                    0    2019-01-11  2019-01-11
1023                   0    2019-01-09  2019-01-09
1338                   0    2019-01-29  2019-01-29
1379                   0    2019-02-15  2019-02-15
1519                   0    2019-01-29  2019-01-29
1534                   0    2019-02-06  2019-02-06
1582                   0    2019-01-31  2019-01-31
1635                   0    2019-02-08  2019-02-08
1716                   0    2019-02-01  2019-02-01
1763                   0    2019-02-04  2019-02-04
1765                   0    2019-02-01  2019-02-01
1767                   0    2019-02-01  2019-02-01
2167                   0    2019-03-09  2019-03-09
2216                   0    2019-02-19  2019-02-19
2347                   0    2019-02-26  2019-02-26
2358                   0    2019-02-22  2019-02-22
2400                   0    2019-02-28  2019-02-28
2558                   0    2019-02-28  2019-02-28
2580                   0    2019-03-11  2019-03-11
2623                   0    2019-02-28  2019-02-28
...                  ...           ...         ...
9602                   0    2019-09-23  2019-09-23
9739                   0    2019-10-14  2019-10-14
10104                  0    2019-09-30  2019-09-30
10368                  0    2019-10-21  2019-10-21
10380                  0    2019-10-22  2019-10-22
10492                  0    2019-10-19  2019-10-19
10633                  0    2019-10-24  2019-10-24
10651                  0    2019-10-21  2019-10-21
10653                  0    2019-11-03  2019-11-03
10729                  0    2019-10-28  2019-10-28
10781                  0    2019-10-29  2019-10-29
10805                  0    2019-10-22  2019-10-22
10806                  0    2019-10-22  2019-10-22
10859                  0    2019-10-25  2019-10-25
10900                  0    2019-11-04  2019-11-04
10924                  0    2019-10-28  2019-10-28
11019                  0    2019-11-11  2019-11-11
11146                  0    2019-11-12  2019-11-12
11326                  0    2019-11-15  2019-11-15
11443                  0    2019-11-13  2019-11-13
11712                  0    2019-11-27  2019-11-27
11749                  0    2019-11-15  2019-11-15
11751                  0    2019-11-18  2019-11-18
11991                  0    2019-12-09  2019-12-09
12132                  0    2019-11-15  2019-11-15
12154                  0    2019-11-28  2019-11-28
12236                  0    2019-12-06  2019-12-06
12264                  0    2019-12-04  2019-12-04
12358                  0    2019-12-16  2019-12-16
12485                  0    2019-12-09  2019-12-09

[115 rows x 3 columns]

In [27]:
df_s_6.shape

(12723, 27)

In [3]:
df_m_2 = pd.read_pickle('df_m_2.pkl')

df_omr = df_m_2[['SAPOrderNo','Material#','region','grade','ContractStart']]

df_a_3 = pd.read_pickle('df_a_3.pkl')

df_omr_2 = pd.merge(df_omr,df_a_3[['Quote date', 'quote', 'region', 'Grade ']],
                  how='left',left_on=['ContractStart','grade','region'],right_on=['Quote date','Grade ','region'])

In [8]:
df_omr_2.groupby('SAPOrderNo')['Grade '].apply(lambda x : '_'.join(x.tolist()))
df_omr_2['Grade_m'] = df_omr_2.groupby('SAPOrderNo')['Grade '].transform(lambda x : '_'.join(x.tolist()))

SAPOrderNo
4001331196                   HO
4001332133                   HO
4001332166                  ADO
4001332268                   HO
4001332269                   HO
4001332270                   HO
4001332271                  ADO
4001332272                   HO
4001332273                   HO
4001332274                   HO
4001332275                   HO
4001332276                  ADO
4001332277                   HO
4001332278                   HO
4001332279                  ADO
4001332280                  ADO
4001332281                   HO
4001332282                   HO
4001332283                   HO
4001332284                   HO
4001332285                   HO
4001332286                   HO
4001332287                  ADO
4001332288                   HO
4001332289                   HO
4001332290                  ADO
4001332291                   HO
4001332292                  ADO
4001332294                  ADO
4001332295                  ADO
                    ...      

In [11]:
df_omr_2['Grade_m'].shape

(14006,)

In [14]:
df_omr_2 = df_omr_2.drop_duplicates(subset = ['SAPOrderNo','Material#'])

In [15]:
df_omr_2.shape

(14006, 9)

In [ ]:
# df_omr_2.isnull().sum()

# df_omr_2.shape

df_omr_2 = df_omr_2.drop_duplicates(subset = ['SAPOrderNo','Material#'])

# df_omr_2.columns

df_omr_2.rename(columns={'quote':'contract_st_omr_qt'},inplace=True)